- use_log: 2018年度ジム利用履歴
- customer_master: 2019/03時点の会員データ
- class_master: 会員区分
- campaign_master: キャンペーン区分データ
## 21 データを読み込んで把握

In [3]:
import pandas as pd
# ジム利用履歴データ
uselog = pd.read_csv('./assets/3章/use_log.csv')
print(len(uselog))
uselog.head()

197428


,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


In [4]:
# 会員データ
customer = pd.read_csv('./assets/3章/customer_master.csv')
print(len(customer))
customer.head()

4192


,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0


In [6]:
# 会員区分データ
class_master = pd.read_csv('./assets/3章/class_master.csv')
print(len(class_master))
class_master

3


,class,class_name,price
0,C01,オールタイム,10500
1,C02,デイタイム,7500
2,C03,ナイト,6000


In [7]:
# キャンペーン区分データ
campaign_master = pd.read_csv('./assets/3章/campaign_master.csv')
print(len(campaign_master))
campaign_master

3


,campaign_id,campaign_name
0,CA1,通常
1,CA2,入会費半額
2,CA3,入会費無料


## 顧客データの整形

In [21]:
# 顧客データと各データをマージする
# classをキーとして会員区分をleftjoin
customer_join = pd.merge(customer, class_master, on="class", how="left")
# campaign_idをキーとしてキャンペーン区分をleftjoin
customer_join = pd.merge(customer_join, campaign_master, on="campaign_id", how="left")
# データ数の検証、データ表示
display(len(customer_join)==len(customer), customer_join.head())

True

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0,ナイト,6000,通常


In [24]:
# 欠損値の個数を確認
customer_join.isnull().sum()
# 退会していないユーザはnullになってるが、それ以外は欠損なし

customer_id         0
name                0
class               0
gender              0
start_date          0
end_date         2842
campaign_id         0
is_deleted          0
class_name          0
price               0
campaign_name       0
dtype: int64

## 顧客データの基礎集計

In [31]:
# とりま、てきとうにグループ分けして集計してみる
display("会員区分ごと", customer_join.groupby("class_name").count()["customer_id"])
display("キャンペーン区分ごと", customer_join.groupby("campaign_id").count()["customer_id"])
display("性別ごと", customer_join.groupby("gender").count()["customer_id"])
display("退会有無ごと", customer_join.groupby("is_deleted").count()["customer_id"])

'会員区分ごと'

class_name
オールタイム    2045
デイタイム     1019
ナイト       1128
Name: customer_id, dtype: int64

'キャンペーン区分ごと'

campaign_id
CA1    3050
CA2     650
CA3     492
Name: customer_id, dtype: int64

'性別ごと'

gender
F    1983
M    2209
Name: customer_id, dtype: int64

'退会有無ごと'

is_deleted
0    2842
1    1350
Name: customer_id, dtype: int64

In [38]:
# 入会日年月を再フォーマット
customer_join["start_date"] = pd.to_datetime(customer_join["start_date"])
# 2018年度の入会者数をカウント
customer_start = customer_join.loc[customer_join["start_date"] > pd.to_datetime('2018/04/01')]
len(customer_start)

1361

## 最新顧客データの基礎集計